In [35]:
before = pd.read_csv("driving_baseline.csv")
after = pd.read_csv("driving_mod.csv")
compare_out_file = "driving_mod_compare.csv"

In [36]:
df = pd.merge(before, after, on="Name", suffixes=("_before", "_after"))

In [37]:
df['diff_hh'] = df['weighted_hh_after'] - df['weighted_hh_before']
df['diff_jobs'] = df['weighted_jobs_after'] - df['weighted_jobs_before']
df['diff_ato'] = df['ato_after'] - df['ato_before']

In [38]:
df.groupby('Origin_CO_TAZID_before')['diff_ato'].sum()

Origin_CO_TAZID_before
30001     0.0
30002     0.0
30003     0.0
30004     0.0
30005     0.0
         ... 
570416    5.0
570417    2.0
570418    5.0
570419    5.0
570420    2.0
Name: diff_ato, Length: 2858, dtype: float64

In [39]:
df.head()

,Unnamed: 0_before,Name,Origin_CO_TAZID_before,Dest_CO_TAZID_before,travel_time_before,survey_weight_before,weighted_jobs_before,weighted_hh_before,ato_before,Unnamed: 0_after,Origin_CO_TAZID_after,Dest_CO_TAZID_after,travel_time_after,survey_weight_after,weighted_jobs_after,weighted_hh_after,ato_after,diff_hh,diff_jobs,diff_ato
0,0,491896 - 491896,491896,491896,0.000000,1.000,287.0,0.0,287.0,0,491896,491896,0.000000,1.000,287.0,0.0,287.0,0.0,0.0,0.0
1,1,492490 - 491896,492490,491896,34.753781,0.097,28.0,0.0,28.0,1,492490,491896,34.753781,0.097,28.0,0.0,28.0,0.0,0.0,0.0
2,2,492493 - 491896,492493,491896,36.209782,0.084,24.0,0.0,24.0,2,492493,491896,36.209782,0.084,24.0,0.0,24.0,0.0,0.0,0.0
3,3,492632 - 491896,492632,491896,35.391817,0.091,26.0,0.0,26.0,3,492632,491896,35.391817,0.091,26.0,0.0,26.0,0.0,0.0,0.0
4,4,491859 - 491896,491859,491896,9.474881,0.767,220.0,0.0,220.0,4,491859,491896,9.474881,0.767,220.0,0.0,220.0,0.0,0.0,0.0


In [40]:
assert df['diff_ato'].min() >= 0

AssertionError: 

In [41]:
summary = df.groupby('Origin_CO_TAZID_before').agg(
    jobs=pd.NamedAgg(column='diff_jobs', aggfunc=sum),
    hh=pd.NamedAgg(column='diff_hh', aggfunc=sum),
    ato=pd.NamedAgg(column='diff_ato', aggfunc=sum))

In [42]:
summary.to_csv(compare_out_file)

In [43]:
df.agg(
    jobs=pd.NamedAgg(column='diff_jobs', aggfunc=sum),
    hh=pd.NamedAgg(column='diff_hh', aggfunc=sum),
    ato=pd.NamedAgg(column='diff_ato', aggfunc=sum))

,diff_jobs,diff_hh,diff_ato
jobs,202287.0,NaN,NaN
hh,NaN,154537.0,NaN
ato,NaN,NaN,356824.0
